In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
import torch
from torch.nn import BCEWithLogitsLoss, BCELoss
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from keras_preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.metrics import classification_report, confusion_matrix, multilabel_confusion_matrix, f1_score, accuracy_score
import pickle
from transformers import *
from tqdm import tqdm, trange
from ast import literal_eval

2022-08-28 12:15:04.989542: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-08-28 12:15:04.989586: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2022-08-28 12:15:11.568766: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2022-08-28 12:15:11.568816: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2022-08-28 12:15:11.568838: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (noota-nlp-v4-nogpu): /proc/driver/nvidia/version does not exist
2022-08-28 12:15:11.569607: I tensorflow/core/platform/cpu

In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cpu')

In [3]:
num_labels = 10

In [4]:
df = pd.read_csv("./data/cleaned_data.csv")
df.head()

,full_text,aac,aapt,aar,abaddressbook,abi,abort,absolutepath,absolutevalue,abstract,...,zooming,zorder,zpl,zplii,zsh,zshcompletion,zshrc,zsi,zurbfoundation,zxing
0,Pan & Zoom Image\nI want to create a simple im...,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
1,Is a bool read/write atomic in C#\nIs accessin...,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,What is the advantage of storing schema in avr...,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,How do you get JavaScript/jQuery Intellisense ...,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,XmlSerializer - There was an error reflecting ...,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [5]:
# TODO: Word2Vec :: Glove - Fastext
def get_top_n(dataframe, top_n):
    """
    Get the top n of the created 
    """
    categories = list(dataframe.columns.values)[1:]
    df_stats = pd.DataFrame([(category, dataframe[category].sum()) for category in categories], columns=['category', 'number of queries'])
    df_stats.sort_values(by=['number of queries'], ascending = False, inplace=True)
    
    top_ = df_stats['category'][:top_n].tolist()
    df_top = dataframe[['full_text'] + top_]
    
    # Remove raws without labels
    df_top = df_top[(df_top[top_].T != 0).any()]
    df_top = df_top.reset_index(drop=True)
    return df_top, top_

In [6]:
df, label_cols = get_top_n(df, num_labels)
df['one_hot_labels'] = list(df[label_cols].values)
df.head()

,full_text,java,python,javascript,ios,android,iphone,objectivec,html,jquery,php,one_hot_labels
0,What is the advantage of storing schema in avr...,1,0,0,0,0,0,0,0,0,0,"[1, 0, 0, 0, 0, 0, 0, 0, 0, 0]"
1,How do you get JavaScript/jQuery Intellisense ...,0,0,1,0,0,0,0,0,1,0,"[0, 0, 1, 0, 0, 0, 0, 0, 1, 0]"
2,How can I prevent SQL injection in PHP?\nIf us...,0,0,0,0,0,0,0,0,0,1,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 1]"
3,Mocking Static Blocks in Java\nMy motto for Ja...,1,0,0,0,0,0,0,0,0,0,"[1, 0, 0, 0, 0, 0, 0, 0, 0, 0]"
4,Getting random row through SQLAlchemy\nHow do ...,0,1,0,0,0,0,0,0,0,0,"[0, 1, 0, 0, 0, 0, 0, 0, 0, 0]"


In [7]:
labels = list(df.one_hot_labels.values)
comments = list(df.full_text.values)

In [9]:
max_length = 512
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True) # tokenizer
encodings = tokenizer.batch_encode_plus(comments, max_length=max_length, padding='max_length', truncation=True) # tokenizer's encoding method
input_ids = encodings['input_ids'] # tokenized and encoded sentences
token_type_ids = encodings['token_type_ids'] # token type ids
attention_masks = encodings['attention_mask'] # attention masks

print('tokenizer outputs: ', encodings.keys())

loading file https://huggingface.co/bert-base-uncased/resolve/main/vocab.txt from cache at /home/jupyter/.cache/huggingface/transformers/45c3f7a79a80e1cf0a489e5c62b43f173c15db47864303a55d623bb3c96f72a5.d789d64ebfe299b0e416afc4a169632f903f693095b4629a7ea271d5a0cf2c99
loading file https://huggingface.co/bert-base-uncased/resolve/main/added_tokens.json from cache at None
loading file https://huggingface.co/bert-base-uncased/resolve/main/special_tokens_map.json from cache at None
loading file https://huggingface.co/bert-base-uncased/resolve/main/tokenizer_config.json from cache at /home/jupyter/.cache/huggingface/transformers/c1d7f0a763fb63861cc08553866f1fc3e5a6f4f07621be277452d26d71303b7e.20430bd8e10ef77a7d2977accefe796051e01bc2fc4aa146bc862997a1a15e79
loading configuration file https://huggingface.co/bert-base-uncased/resolve/main/config.json from cache at /home/jupyter/.cache/huggingface/transformers/3c61d016573b14f7f008c02c4e51a366c67ab274726fe2910691e2a761acf43e.37395cee442ab11005bcd2

tokenizer outputs:  dict_keys(['input_ids', 'token_type_ids', 'attention_mask'])


In [10]:
# Identifying indices of 'one_hot_labels' entries that only occur once - this will allow us to stratify split our training data later
label_counts = df.one_hot_labels.astype(str).value_counts()
one_freq = label_counts[label_counts==1].keys()
one_freq_idxs = sorted(list(df[df.one_hot_labels.astype(str).isin(one_freq)].index), reverse=True)
print('df label indices with only one instance: ', one_freq_idxs)

df label indices with only one instance:  [13940, 13441, 12061, 11554, 11530, 11454, 10736, 10597, 9109, 8889, 8166, 8011, 8004, 7680, 7439, 6976, 6661, 6441, 5909, 5320, 5275, 5133, 5033, 4676, 4420, 3851, 2895, 2220, 1978, 1468, 1269]


In [11]:
one_freq_idxs

[13940,
 13441,
 12061,
 11554,
 11530,
 11454,
 10736,
 10597,
 9109,
 8889,
 8166,
 8011,
 8004,
 7680,
 7439,
 6976,
 6661,
 6441,
 5909,
 5320,
 5275,
 5133,
 5033,
 4676,
 4420,
 3851,
 2895,
 2220,
 1978,
 1468,
 1269]

In [12]:
# Gathering single instance inputs to force into the training set after stratified split
one_freq_input_ids = [input_ids.pop(i) for i in one_freq_idxs]
one_freq_token_types = [token_type_ids.pop(i) for i in one_freq_idxs]
one_freq_attention_masks = [attention_masks.pop(i) for i in one_freq_idxs]
one_freq_labels = [labels.pop(i) for i in one_freq_idxs]

In [13]:
# Use train_test_split to split our data into train and validation sets
train_inputs, validation_inputs, train_labels, validation_labels, train_token_types, validation_token_types, train_masks, validation_masks = train_test_split(input_ids, labels, token_type_ids, attention_masks,
                                                            random_state=2020, test_size=0.10, stratify = labels)

# Add one frequency data to train data
train_inputs.extend(one_freq_input_ids)
train_labels.extend(one_freq_labels)
train_masks.extend(one_freq_attention_masks)
train_token_types.extend(one_freq_token_types)

# Convert all of our data into torch tensors, the required datatype for our model
train_inputs = torch.tensor(train_inputs)
train_labels = torch.tensor(train_labels)
train_masks = torch.tensor(train_masks)
train_token_types = torch.tensor(train_token_types)

validation_inputs = torch.tensor(validation_inputs)
validation_labels = torch.tensor(validation_labels)
validation_masks = torch.tensor(validation_masks)
validation_token_types = torch.tensor(validation_token_types)

/tmp/ipykernel_10210/3411898742.py:13: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at  ../torch/csrc/utils/tensor_new.cpp:204.)
  train_labels = torch.tensor(train_labels)


In [14]:
# Select a batch size for training. For fine-tuning with XLNet, the authors recommend a batch size of 32, 48, or 128. We will use 32 here to avoid memory issues.
batch_size = 32

# Create an iterator of our data with torch DataLoader. This helps save on memory during training because, unlike a for loop, 
# with an iterator the entire dataset does not need to be loaded into memory

train_data = TensorDataset(train_inputs, train_masks, train_labels, train_token_types)
train_sampler = RandomSampler(train_data)
train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=batch_size)

validation_data = TensorDataset(validation_inputs, validation_masks, validation_labels, validation_token_types)
validation_sampler = SequentialSampler(validation_data)
validation_dataloader = DataLoader(validation_data, sampler=validation_sampler, batch_size=batch_size)

In [14]:
torch.save(validation_dataloader,'validation_data_loader')
torch.save(train_dataloader,'train_data_loader')

In [15]:
# Load model, the pretrained model will include a single linear classification layer on top for classification. 
model = BertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=num_labels)
model.to(device)

loading configuration file https://huggingface.co/bert-base-uncased/resolve/main/config.json from cache at /home/jupyter/.cache/huggingface/transformers/3c61d016573b14f7f008c02c4e51a366c67ab274726fe2910691e2a761acf43e.37395cee442ab11005bcd270f3c34464dc1704b715b5d7d52b1a461abe3b9e4e
Model config BertConfig {
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2",
    "3": "LABEL_3",
    "4": "LABEL_4",
    "5": "LABEL_5",
    "6": "LABEL_6",
    "7": "LABEL_7",
    "8": "LABEL_8",
    "9": "LABEL_9"
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "label2id": {
    "LABEL_0": 0,
    "LABEL_1": 1,
    "LABEL_2": 2,
    "LABEL_3": 3,
    "LABEL_4": 4,
    "LABEL_5": 5,
    "LABEL_6": 6,
    "LABEL_7": 7,
    "LABEL_8": 8,
    "LA

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, element

In [16]:
# setting custom optimization parameters. You may implement a scheduler here as well.
param_optimizer = list(model.named_parameters())
no_decay = ['bias', 'gamma', 'beta']
optimizer_grouped_parameters = [
    {'params': [p for n, p in param_optimizer if not any(nd in n for nd in no_decay)],
     'weight_decay_rate': 0.01},
    {'params': [p for n, p in param_optimizer if any(nd in n for nd in no_decay)],
     'weight_decay_rate': 0.0}
]

In [17]:
optimizer = AdamW(optimizer_grouped_parameters,lr=2e-5,correct_bias=True)
# optimizer = AdamW(model.parameters(),lr=2e-5)  # Default optimization

/home/jupyter/.cache/pypoetry/virtualenvs/openclassroom-KKh1tyRT-py3.9/lib/python3.9/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


# Training model :

In [ ]:
# Store our loss and accuracy for plotting
train_loss_set = []

# Number of training epochs (authors recommend between 2 and 4)
epochs = 3

# trange is a tqdm wrapper around the normal python range
for _ in trange(epochs, desc="Epoch"):
    # Training

    # Set our model to training mode (as opposed to evaluation mode)
    model.train()

    # Tracking variables
    tr_loss = 0 #running loss
    nb_tr_examples, nb_tr_steps = 0, 0
  
  # Train the data for one epoch
    for step, batch in enumerate(train_dataloader):
        # Add batch to GPU
        batch = tuple(t.to(device) for t in batch)
        # Unpack the inputs from our dataloader
        b_input_ids, b_input_mask, b_labels, b_token_types = batch
        # Clear out the gradients (by default they accumulate)
        optimizer.zero_grad()

        # # Forward pass for multiclass classification
        # outputs = model(b_input_ids, token_type_ids=None, attention_mask=b_input_mask, labels=b_labels)
        # loss = outputs[0]
        # logits = outputs[1]

        # Forward pass for multilabel classification
        outputs = model(b_input_ids, token_type_ids=None, attention_mask=b_input_mask)
        logits = outputs[0]
        loss_func = BCEWithLogitsLoss() 
        loss = loss_func(logits.view(-1,num_labels),b_labels.type_as(logits).view(-1,num_labels)) #convert labels to float for calculation
        # loss_func = BCELoss() 
        # loss = loss_func(torch.sigmoid(logits.view(-1,num_labels)),b_labels.type_as(logits).view(-1,num_labels)) #convert labels to float for calculation
        train_loss_set.append(loss.item())    

        # Backward pass
        loss.backward()
        # Update parameters and take a step using the computed gradient
        optimizer.step()
        # scheduler.step()
        # Update tracking variables
        tr_loss += loss.item()
        nb_tr_examples += b_input_ids.size(0)
        nb_tr_steps += 1

    print("Train loss: {}".format(tr_loss/nb_tr_steps))

###############################################################################

    # Validation

    # Put model in evaluation mode to evaluate loss on the validation set
    model.eval()

    # Variables to gather full output
    logit_preds,true_labels,pred_labels,tokenized_texts = [],[],[],[]

    # Predict
    for i, batch in enumerate(validation_dataloader):
        batch = tuple(t.to(device) for t in batch)
        # Unpack the inputs from our dataloader
        b_input_ids, b_input_mask, b_labels, b_token_types = batch
        with torch.no_grad():
            # Forward pass
            outs = model(b_input_ids, token_type_ids=None, attention_mask=b_input_mask)
            b_logit_pred = outs[0]
            pred_label = torch.sigmoid(b_logit_pred)

            b_logit_pred = b_logit_pred.detach().cpu().numpy()
            pred_label = pred_label.to('cpu').numpy()
            b_labels = b_labels.to('cpu').numpy()

        tokenized_texts.append(b_input_ids)
        logit_preds.append(b_logit_pred)
        true_labels.append(b_labels)
        pred_labels.append(pred_label)

    # Flatten outputs
    pred_labels = [item for sublist in pred_labels for item in sublist]
    true_labels = [item for sublist in true_labels for item in sublist]

    # Calculate Accuracy
    threshold = 0.50
    pred_bools = [pl>threshold for pl in pred_labels]
    true_bools = [tl==1 for tl in true_labels]
    val_f1_accuracy = f1_score(true_bools,pred_bools,average='micro')*100
    val_flat_accuracy = accuracy_score(true_bools, pred_bools)*100

    print('F1 Validation Accuracy: ', val_f1_accuracy)
    print('Flat Validation Accuracy: ', val_flat_accuracy)

Epoch:   0%|          | 0/3 [00:00<?, ?it/s]

In [ ]:
torch.save(model.state_dict(), 'bert_model_stackoverflow10')

In [8]:
model_eval = BertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=num_labels)
model_eval.load_state_dict(torch.load("bert_model_stackoverflow10",map_location='cpu'))
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)

loading configuration file https://huggingface.co/bert-base-uncased/resolve/main/config.json from cache at /home/jupyter/.cache/huggingface/transformers/3c61d016573b14f7f008c02c4e51a366c67ab274726fe2910691e2a761acf43e.37395cee442ab11005bcd270f3c34464dc1704b715b5d7d52b1a461abe3b9e4e
Model config BertConfig {
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2",
    "3": "LABEL_3",
    "4": "LABEL_4",
    "5": "LABEL_5",
    "6": "LABEL_6",
    "7": "LABEL_7",
    "8": "LABEL_8",
    "9": "LABEL_9"
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "label2id": {
    "LABEL_0": 0,
    "LABEL_1": 1,
    "LABEL_2": 2,
    "LABEL_3": 3,
    "LABEL_4": 4,
    "LABEL_5": 5,
    "LABEL_6": 6,
    "LABEL_7": 7,
    "LABEL_8": 8,
    "LA

In [9]:
config = AutoConfig.from_pretrained("bert-base-uncased")
print('Architecture:', config.architectures)
print('Classes:', config.label2id.keys())

loading configuration file https://huggingface.co/bert-base-uncased/resolve/main/config.json from cache at /home/jupyter/.cache/huggingface/transformers/3c61d016573b14f7f008c02c4e51a366c67ab274726fe2910691e2a761acf43e.37395cee442ab11005bcd270f3c34464dc1704b715b5d7d52b1a461abe3b9e4e
Model config BertConfig {
  "_name_or_path": "bert-base-uncased",
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.19.4",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 30522
}



Architecture: ['BertForMaskedLM']
Classes: dict_keys(['LABEL_0', 'LABEL_1'])


In [10]:
sample = list(df['full_text'].values)[:5]
output = list(df['one_hot_labels'].values)[:5]

In [11]:
for x in output:
    print(list(x))

[1, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 1, 0, 0, 0, 0, 0, 1, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 1]
[1, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 1, 0, 0, 0, 0, 0, 0, 0, 0]


In [12]:
inputs = tokenizer(sample, padding=True, return_tensors='pt')
predictions = model_eval(**inputs)
predictions

SequenceClassifierOutput(loss=None, logits=tensor([[-1.0403, -1.1314, -1.0400, -1.4449, -1.3802, -1.6507, -1.2042, -1.6691,
         -1.5493, -1.4334],
        [-1.1415, -1.2116, -1.0174, -1.2311, -1.2393, -1.5144, -1.3089, -1.5642,
         -1.5716, -1.4092],
        [-0.8805, -1.1731, -1.0730, -1.4312, -1.5033, -1.6267, -1.3377, -1.6562,
         -1.3694, -1.3657],
        [-0.9307, -0.9927, -0.9412, -1.5989, -1.3920, -1.7090, -1.0253, -1.6602,
         -1.5600, -1.4340],
        [-0.8835, -1.2383, -1.1141, -1.1727, -1.4453, -1.4696, -1.4882, -1.5309,
         -1.2114, -1.2073]], grad_fn=<AddmmBackward0>), hidden_states=None, attentions=None)

In [13]:
import torch
probs = torch.nn.Softmax(dim=1)(predictions.logits)

In [14]:
probs = probs.detach().numpy()
probs

array([[0.13344575, 0.12183134, 0.13348427, 0.08904416, 0.09499041,
        0.07247756, 0.11327576, 0.07115609, 0.08021996, 0.09007474],
       [0.11777221, 0.10979938, 0.13332842, 0.10767699, 0.10680063,
        0.08111198, 0.09961499, 0.07717503, 0.07660509, 0.0901153 ],
       [0.15430231, 0.11516006, 0.12728071, 0.08896305, 0.08277246,
        0.07316962, 0.09768192, 0.07104174, 0.09464041, 0.09498768],
       [0.14161526, 0.13311276, 0.14014557, 0.07259672, 0.08928852,
        0.06503148, 0.12883416, 0.06828495, 0.07547785, 0.08561274],
       [0.14525962, 0.10186779, 0.11533957, 0.10877216, 0.08282328,
        0.08083776, 0.07934622, 0.07602552, 0.10465296, 0.10507512]],
      dtype=float32)

In [15]:
classes = probs.argmax(axis=1)
confidences = probs.max(axis=1)

In [16]:
classes

array([2, 2, 0, 0, 0])

In [17]:
confidences

array([0.13348427, 0.13332842, 0.15430231, 0.14161526, 0.14525962],
      dtype=float32)

In [18]:
label_cols

['java',
 'python',
 'javascript',
 'ios',
 'android',
 'iphone',
 'objectivec',
 'html',
 'jquery',
 'php']

In [19]:
outputs = pd.DataFrame({ 'rating': [label_cols[c] for c in classes], 'confidence': confidences })
outputs.to_csv('outputs.csv')

In [20]:
from mlflow.models.signature import infer_signature

In [21]:
sample = pd.DataFrame({ 'text': sample})

sample

,text
0,What is the advantage of storing schema in avr...
1,How do you get JavaScript/jQuery Intellisense ...
2,How can I prevent SQL injection in PHP?\nIf us...
3,Mocking Static Blocks in Java\nMy motto for Ja...
4,Getting random row through SQLAlchemy\nHow do ...


In [22]:
signature = infer_signature(sample, outputs)
type(signature)

mlflow.models.signature.ModelSignature

In [23]:
model_path = "./models/bert_Stackoverflow10"
model_eval.save_pretrained(model_path)
tokenizer.save_pretrained(model_path)

Configuration saved in ./models/bert_Stackoverflow10/config.json
Model weights saved in ./models/bert_Stackoverflow10/pytorch_model.bin
tokenizer config file saved in ./models/bert_Stackoverflow10/tokenizer_config.json
Special tokens file saved in ./models/bert_Stackoverflow10/special_tokens_map.json


('./models/bert_Stackoverflow10/tokenizer_config.json',
 './models/bert_Stackoverflow10/special_tokens_map.json',
 './models/bert_Stackoverflow10/vocab.txt',
 './models/bert_Stackoverflow10/added_tokens.json')

In [24]:
import os, pathlib

artifacts = { pathlib.Path(file).stem: os.path.join(model_path, file) 
             for file in os.listdir(model_path) 
             if not os.path.basename(file).startswith('.') }

In [25]:
artifacts

{'vocab': './models/bert_Stackoverflow10/vocab.txt',
 'tokenizer_config': './models/bert_Stackoverflow10/tokenizer_config.json',
 'special_tokens_map': './models/bert_Stackoverflow10/special_tokens_map.json',
 'pytorch_model': './models/bert_Stackoverflow10/pytorch_model.bin',
 'config': './models/bert_Stackoverflow10/config.json'}

In [38]:
from mlflow.pyfunc import PythonModel, PythonModelContext
from typing import Dict

class BertTextClassifier(PythonModel):
    def load_context(self, context: PythonModelContext):
        import os
        from transformers import BertConfig
        from transformers import BertForSequenceClassification
        from transformers import BertTokenizer
        self.model = BertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=10)
        self.model.load_state_dict(torch.load("bert_model_stackoverflow10", map_location='cpu'))
        self.tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)
        self.labels = ['java','python','javascript','ios','android','iphone','objectivec','html','jquery','php']
        
        if torch.cuda.is_available():
            print('[INFO] Model is being sent to CUDA device as GPU is available')
            self.model = self.model.cuda()
        else:
            print('[INFO] Model will use CPU runtime')
        
        _ = self.model.eval()
        
    def _predict_batch(self, data):
        import torch
        import pandas as pd
        
        with torch.no_grad():
            inputs = self.tokenizer(list(data['text'].values), padding=True, return_tensors='pt', truncation=True)
        
            if self.model.device.index != None:
                torch.cuda.empty_cache()
                for key in inputs.keys():
                    inputs[key] = inputs[key].to(self.model.device.index)

            predictions = self.model(**inputs)
            
            probs = torch.nn.Softmax(dim=1)(predictions.logits)
            probs = probs.detach().cpu().numpy()
            
            classes = probs.argmax(axis=1)
            confidences = probs.max(axis=1)

            return classes, confidences
        
    def predict(self, context: PythonModelContext, data: pd.DataFrame) -> pd.DataFrame:
        import math
        import numpy as np
        
        batch_size = 64
        sample_size = len(data)
        
        classes = np.zeros(sample_size)
        confidences = np.zeros(sample_size)

        for batch_idx in range(0, math.ceil(sample_size / batch_size)):
            bfrom = batch_idx * batch_size
            bto = bfrom + batch_size
            
            c, p = self._predict_batch(data.iloc[bfrom:bto])
            classes[bfrom:bto] = c
            confidences[bfrom:bto] = p
            
        return pd.DataFrame({'rating': [self.labels[c] for c in classes], 
                             'confidence': confidences})

In [39]:
import mlflow

mlflow.set_tracking_uri("http://localhost:5000")
mlflow.set_experiment('bert-classification')

with mlflow.start_run() as run:
    mlflow.pyfunc.log_model('classifier', 
                            python_model=BertTextClassifier(), 
                            artifacts=None, 
                            signature=signature,
                            registered_model_name='bert-stackoverflow-classification')

Registered model 'bert-stackoverflow-classification' already exists. Creating a new version of this model...
2022/08/28 12:54:44 INFO mlflow.tracking._model_registry.client: Waiting up to 300 seconds for model version to finish creation.                     Model name: bert-stackoverflow-classification, version 2
Created version '2' of model 'bert-stackoverflow-classification'.


In [40]:
import mlflow
model = mlflow.pyfunc.load_model('models:/bert-stackoverflow-classification/latest')

loading configuration file https://huggingface.co/bert-base-uncased/resolve/main/config.json from cache at /home/jupyter/.cache/huggingface/transformers/3c61d016573b14f7f008c02c4e51a366c67ab274726fe2910691e2a761acf43e.37395cee442ab11005bcd270f3c34464dc1704b715b5d7d52b1a461abe3b9e4e
Model config BertConfig {
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2",
    "3": "LABEL_3",
    "4": "LABEL_4",
    "5": "LABEL_5",
    "6": "LABEL_6",
    "7": "LABEL_7",
    "8": "LABEL_8",
    "9": "LABEL_9"
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "label2id": {
    "LABEL_0": 0,
    "LABEL_1": 1,
    "LABEL_2": 2,
    "LABEL_3": 3,
    "LABEL_4": 4,
    "LABEL_5": 5,
    "LABEL_6": 6,
    "LABEL_7": 7,
    "LABEL_8": 8,
    "LA

[INFO] Model will use CPU runtime
